<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# <strong> Important </strong>

For this assignment, as you did in the first lab, do not use the native fuctions <code>map()</code> and <code>reduce()</code> of Python.


<hr>



## Flights and Airports Data

For this assignment you are going to analyze a dataset that will include flight data.

The dataset comes from <a href="https://www.kaggle.com/flashgordon/usa-airport-dataset">Kaggle</a> and is presented here as an extract called <code>extract_flights.csv</code>

The dataset is in a <code>.csv</code> file. Each line of the file represents a different flight. The data collected contain:


<code>Origin_airport</code>: Three letter airport code of the origin airport </br>
<code>Destination_airport</code>: Three letter airport code of the destination airport</br>
<code>Origin_city</code>: Origin city name</br>
<code>Destination_city</code>: Destination city name</br>
<code>Passengers</code>: Number of passengers transported from origin to destination</br>
<code>Seats</code>: Number of seats available on flights from origin to destination</br>
<code>Flights</code>: Number of flights between origin and destination (multiple records for one month, many with flights > 1)</br>
<code>Distance</code>: Distance (to nearest mile) flown between origin and destination</br>
<code>Fly_date</code>: The date (yyyymm) of flight</br>
<code>Origin_population</code>: Origin city's population as reported by US Census</br>
<code>Destination_population</code>: Destination city's population as reported by US Census</br>


Two files are available in the folder:

<ul>
  <li> <code> small_file.csv</code> </li>
  <li> <code> medium_file.csv</code> </li>
</ul>

For more complex analysis/statistics you can also download the original file available on kaggle.



### Helper function ... just in case

Here is a function that you can modify, reuse, not-use for giving you an idea about how to read a <code>.csv</code> file and remove the header.

In [81]:
# Here a simple function that reads a .csv file, removes the header,
# and writes the output on a .txt file having the same name of the input file + the keyword "no_header"
# input: a csv file
def remove_header(filename) :
    with open(filename + ".csv",'r') as f:
        with open(filename + "no_header.txt",'w') as f1:
            next(f) # skip header line
            for line in f:
                f1.write(line)
    return f1

remove_header('/Users/viceroy/Downloads/assignment_1_airports/data/small_file')
print("file correctly saved")


file correctly saved


In [82]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [83]:
#using pandas to visualize/store data
#one could argue that it takes up (local) RAM but it is the same procedureas the above "remove header"
#since the data is still stored in a .txt file on (local) RAM
df = pd.read_csv('/Users/viceroy/Downloads/assignment_1_airports/data/small_file.csv')
df.sample(5)

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
4527,SEA,EKO,"Seattle, WA","Elko, NV",742,868,7,560,1996-08-01,5621538,45034,47.449001,-122.308998,40.824902,-115.792
9611,SLC,EKO,"Salt Lake City, UT","Elko, NV",1632,3540,118,200,2009-11-01,1130293,49603,40.788399,-111.977997,40.824902,-115.792
5646,CRP,EKO,"Corpus Christi, TX","Elko, NV",113,124,1,1375,1998-08-01,409279,48011,27.770399,-97.501198,40.824902,-115.792
6087,OKC,EKO,"Oklahoma City, OK","Elko, NV",121,121,1,1057,1999-08-01,1061524,47319,35.393101,-97.600700,40.824902,-115.792
3856,FSM,EKO,"Fort Smith, AR","Elko, NV",124,124,1,1224,1995-06-01,249759,42918,35.336601,-94.367401,40.824902,-115.792


In [84]:
df_medium = pd.read_csv('/Users/viceroy/Downloads/assignment_1_airports/data/medium_file.csv')
df_medium.sample(5)

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
23723,SFO,RNO,"San Francisco, CA","Reno, NV",2815,4388,34,192,2003-04-01,8306286,374743,37.618999,-122.375000,39.499100,-119.767998
94195,EWR,MIA,"Newark, NJ","Miami, FL",15115,17290,92,1086,2009-07-01,19069796,11094102,40.692501,-74.168701,25.793200,-80.290604
46379,PIR,HON,"Pierre, SD","Huron, SD",246,1007,53,102,2008-05-01,19555,15946,44.382702,-100.286003,44.385201,-98.228500
87823,PHX,MIA,"Phoenix, AZ","Miami, FL",3311,3720,30,1972,2006-03-01,4046571,10933486,33.434299,-112.012001,25.793200,-80.290604
31701,PHX,YUM,"Phoenix, AZ","Yuma, AZ",4127,6669,351,160,1997-01-01,2841395,129070,33.434299,-112.012001,32.656601,-114.606003


### <strong> Exercise 1 - Almost-empty flights</strong> 
#### <strong> 4 points </strong>
Describe and define a MapReduce procedure that gives as output the number of flights that departed with 10% of capacity maximum (i.e. empty, 1%, 6%, etc.) . 

The output can be of any form you like. 

You can use any data structure you want to support your implementation.




In [85]:
#The goal of this map function is to first take the ratio of Passengers/Seats
#then append a pair(index, # of empty flights) if the condition is met (ratio[index] is less than 0.10)
#thus res = a list of flights that had <10% capacity [ (row #, # of empty flights), ..., (row #, # of empty flights)] 
def map_e_flights(df):
    res = []
    ratio = df["Passengers"]/df["Seats"]
    res.append([(i, df["Flights"][i]) for i in df.index if (ratio[i] <= 0.1)])
    return res 

#using res from above, we assign each index to be a key in the default dict, and # of empty flights to be it's value
def shuffle_e_flights(inpt): 
    d = defaultdict(list)
    for array in inpt : 
        for k, v in array:
            d[k].append(v)
    return d 


#finally, going through each key in our default dictionary, we sum the number of empty flights for all keys (and their values)
def reduce_e_flights(d): 
    total = 0
    for key in d:
        total += d[key][0]
    return ("total empty flights", total)

small = reduce_e_flights(
            shuffle_e_flights(
                    map_e_flights( df )))
medium = reduce_e_flights(
            shuffle_e_flights(
                    map_e_flights( df_medium )))
print("Small: ", small)
print("Medium: ", medium)

Small:  ('total empty flights', 1267)
Medium:  ('total empty flights', 13812)


Look at your implementation and think about your program. Is there any option to add a `combine` operation?  If yes describe your new approach and provide an improved version of your code.

Describe in detail your procedure before the code (in the comment lines).

In [106]:
#since there is not several groups of data/yielded/SQL query results, we can not combine any process of our code. 
#This being said, we can use pandas for a much simpler and condensed answer
#df.loc[df["Passengers"]/df["Seats"] <= 0.1]["Flights"].sum()
df_medium.loc[df_medium["Passengers"]/df_medium["Seats"] <= 0.1]["Flights"].sum()

13812

### <strong> Exercise 2 - Top five destination airports </strong>
### <strong> 4 points </strong>

Provide now a function that lists the top five destination <strong>airports</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.



In [87]:
#The goal of this map function is to append a pair(Destination Airport, # of flights) 
#thus res = a list of destination airports and their corresponding number of flights
#[ (Destination Airport, # of flights), ..., (Destination Airport, # of flights)] 
def map_top_flights(df):
    res = []
    res.append([(df["Destination_airport"][i], df["Flights"][i]) for i in df.index])
    return res  #to be completed

#using res from above, we assign each Destination Airport to be a key in the default dict, and # of flights to be it's value
def shuffle_top_flights(inpt): 
    d = defaultdict(list)
    for array in inpt : 
        for k, v in array:
            d[k].append(v)
    return d  #to be completed


#finally, going through each key in our default dictionary, we sum the number of flights for all keys (and their values)
#then sort for only the top 5 (descending order)
def reduce_top_flights(d): 
    res = []
    for k,v in d.items(): #could use d[0], d[1] but k=key, v=val more intuitive
        res.append([k, sum(v)]) #could also use len(v)
    return sorted(res, key =lambda l:l[1], reverse=True)[:5]

small = reduce_top_flights(
            shuffle_top_flights(
                    map_top_flights( df )))
medium = reduce_top_flights(
            shuffle_top_flights(
                    map_top_flights( df_medium )))
print("Small: ", small)
print("Medium: ", medium)

Small:  [['RDM', 108593], ['EKO', 25742], ['ERI', 7101], ['WDG', 3295], ['END', 7]]
Medium:  [['MIA', 1486648], ['RNO', 551046], ['ITO', 201861], ['CAK', 161237], ['FNT', 127985]]


### <strong> Exercise 3 - Top 5 destination cities </strong>
#### <strong>  2 points </strong>

Try to reuse the code you run before and define a function that lists the top five destination <strong>cities</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.


In [88]:
#The goal of this map function is to append a pair(City Airport, # of flights) 
#thus res = a list of destination cities and their corresponding number of flights
#[ (Destination City, # of flights), ..., (Destination City, # of flights)] 
def map_top_cities(df):
    res = []
    res.append([(df["Destination_city"][i], df["Flights"][i]) for i in df.index])
    return res  #to be completed

#using res from above, we assign each destination city to be a key in the default dict, and # of flights to be it's value
def shuffle_top_cities(inpt): 
    d = defaultdict(list)
    for array in inpt : 
        for k, v in array:
            d[k].append(v)
    return d  #to be completed


#finally, going through each key in our default dictionary, we sum the number of flights for all keys (and their values)
def reduce_top_cities(d): 
    res = []
    for k,v in d.items(): #could use d[0], d[1] but k=key, v=val more intuitive
        res.append([k, sum(v)]) #could also use len(v)
    return sorted(res, key =lambda l:l[1], reverse=True)[:5]

small = reduce_top_cities(
            shuffle_top_cities(
                    map_top_cities( df )))
medium = reduce_top_cities(
            shuffle_top_cities(
                    map_top_cities( df_medium )))
print("Small: ", small)
print("Medium: ", medium)

Small:  [['Bend, OR', 108593], ['Elko, NV', 25742], ['Erie, PA', 7101], ['Enid, OK', 3302], ['Ames, IA', 1]]
Medium:  [['Miami, FL', 1487169], ['Reno, NV', 551046], ['Hilo, HI', 201861], ['Akron, OH', 161237], ['Flint, MI', 127985]]


## Statistics on flights

<p align="justify">
<font size="3">
Now we want to run some more complex analysis on the flights. 
</font>
</p>





### <strong> Exercise 4 - Top five connections by month</strong>
#### <strong> 4 points </strong>

Try to reuse the code you run before and define now a function that lists the top five connections by each month: the top five pairs of cities that have the most number of connections (the function takes into account the flights from A to B and from B to A by month/year). Implement an algorithm that uses the MapReduce procedure.

In [110]:
#The goal of this map function is identify the source city, destination city, # of flights and fly date, 
#then append a pair((source, destination, date), # of flights) 
# after rearranging source/destinations to always be alphabetically sorted
#thus res = [ ((source, destination, date), # of flights), ..., ((source, destination, date), # of flights)] 
def map_top_connections(df): #to be completed
    res = []
    for i in df.index:
        source = df["Origin_city"][i]
        destination = df["Destination_city"][i]
        flights = df["Flights"][i]
        date = df["Fly_date"][i][:-3] #we dont want the day numbers
        if source > destination: #sort alphabetically such that Miami -> NYC => NYC -> Miami 
            destination, source = source, destination
        key = (source, destination, date)
        res.append( (key, flights) )
    return res

#using res from above, we assign each (source, destination, date) to be a key in the default dict, and # of flights to be it's value
def shuffle_top_connections(inpt):
    d = defaultdict(list)
    for array in inpt : 
        d[array[0]].append(array[1])
    return d #to be completed


#in this last step of the first map/shuffle/reduce part, we through each key in our default dictionary, 
# then sum the number of flights for all keys (and their values)
def reduce_top_connections(d):
    res = []
    for k,v in d.items(): #could use d[0], d[1] but k=key, v=val more intuitive
        res.append([k, sum(v)]) #could also use len(v)
    return res 

part1_s = reduce_top_connections(
            shuffle_top_connections(
                    map_top_connections( df )))
part1_m = reduce_top_connections(
            shuffle_top_connections(
                    map_top_connections( df_medium )))
#part1 format:
#[((source, destination, date), sum(# of flights)), ..., ((source, destination, date), sum(# of flights))]

In [112]:
### The goal of this second map function reformat the structure of the pairs
#where we now have pair(date, (source, destination, sum(# of flights))) 
#thus res = a list of source,destination pairs with their total flights on the date they are mapped to 
#[(date, (source, destination, sum(# of flights))), ..., (date, (source, destination, sum(# of flights)))]
def map_date(inpt):
    res = []
    for k, v in inpt:
        source, destination, date = k
        val = (source, destination, v)
        res.append( (date, val))
    return res

#using res from above, we assign each date to be a key in the default dict, 
#and (source, destination, sum(# of flights)) to be it's value
def shuffle_date(inpt):
    d = defaultdict(list)
    for array in inpt : 
        d[array[0]].append(array[1])
    return d 

#finally, going through each key in our default dictionary, we sum the number of flights for all date keys (and their values)
def reduce_date(d):
    res = []
    for k,v in d.items(): #could use d[0], d[1] but k=key, v=val more intuitive
        v.sort(key=lambda x: x[2], reverse=True) #descending sort by (source, destination, sum(# of flights))[2] = sum(# of flights)
        res.append((k,v[:5])) #take the top 5
    res.sort() 
    return res 

small = reduce_date(
            shuffle_date(
                    map_date( part1_s ) ))
medium = reduce_date(
            shuffle_date(
                    map_date( part1_m ) ))
print("Small: ", small)
print("\n Medium: ", medium)

Small:  [('1990-01', [('Bend, OR', 'Portland, OR', 184), ('Erie, PA', 'Pittsburgh, PA', 150), ('Bend, OR', 'San Francisco, CA', 86), ('Bend, OR', 'Klamath Falls, OR', 46), ('Erie, PA', 'Ithaca, NY', 30)]), ('1990-02', [('Bend, OR', 'Portland, OR', 176), ('Erie, PA', 'Pittsburgh, PA', 128), ('Bend, OR', 'San Francisco, CA', 83), ('Bend, OR', 'Klamath Falls, OR', 38), ('Erie, PA', 'Ithaca, NY', 24)]), ('1990-03', [('Bend, OR', 'Portland, OR', 203), ('Erie, PA', 'Pittsburgh, PA', 146), ('Bend, OR', 'San Francisco, CA', 92), ('Bend, OR', 'Klamath Falls, OR', 51), ('Erie, PA', 'Ithaca, NY', 29)]), ('1990-04', [('Bend, OR', 'Portland, OR', 197), ('Erie, PA', 'Pittsburgh, PA', 144), ('Bend, OR', 'San Francisco, CA', 83), ('Bend, OR', 'Klamath Falls, OR', 49), ('Erie, PA', 'Ithaca, NY', 28)]), ('1990-05', [('Bend, OR', 'Portland, OR', 205), ('Erie, PA', 'Pittsburgh, PA', 151), ('Bend, OR', 'San Francisco, CA', 88), ('Bend, OR', 'Klamath Falls, OR', 53), ('Erie, PA', 'Ithaca, NY', 1)]), ('1990-

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [ ]:
#since we need to first organize by total flights in a source, destination, date tuple
#then organize by all source, destination, flights in a date, we cannot use combine.
#these are seperate operations that cannot and need not to be combined. 

### <strong> Exercise 5 - Number of full flights</strong>
#### <strong> 2 points </strong>
<p align="justify">
<font size="3">
Describe and implement an algorithm that, following MapReduce procedure, shows how many full flights have departed. This exercise gives you an idea about how many times you can re-use code in MapReduce with minimum effort for repetitive analysis.
</font>
</p>



In [91]:
#The goal of this map function is to first take the ratio of Passengers/Seats
#then append a pair(index, # of full flights) if the condition is met (ratio[index] is equal to 1.0)
#thus res = a list of flights that had 100% capacity [ (row #, # of full flights), ..., (row #, # of full flights)] 
def map_f_flights(df):
    res = []
    ratio = df["Passengers"]/df["Seats"]
    res.append([(i, df["Flights"][i]) for i in df.index if (ratio[i] == 1)])
    return res 

#using res from above, we assign each index to be a key in the default dict, and # of full flights to be it's value
def shuffle_f_flights(inpt): 
    d = defaultdict(list)
    for array in inpt : 
        for k, v in array:
            d[k].append(v)
    return d 

#finally, going through each key in our default dictionary, 
#we sum the number of full flights for all key (and their values)
def reduce_f_flights(d): 
    total = 0
    for key in d:
        total += d[key][0]
    return ("total full flights", total)

small = reduce_f_flights(
            shuffle_f_flights(
                    map_f_flights( df )))
medium = reduce_f_flights(
            shuffle_f_flights(
                    map_f_flights( df_medium )))
print("Small: ", small)
print("Medium: ", medium)

Small:  ('total full flights', 554)
Medium:  ('total full flights', 1470)



### <strong> Exercise 6 -  Average/percentage </strong>
#### <strong> 4 points </strong>

<p align="justify">
<font size="3">
Describe and implement a MapReduce procedure that gives, for each city, the percentage/average, of full flights that have departed.

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before. 
</font>
</p>



In [104]:
#The goal of this map function is to first take the ratio of Passengers/Seats
#then append a pair(Origin City, # of full flights) if the condition is met (ratio[index] is equal to 1.0)
#thus res = a list of flights that had 100% capacity [ (Origin City, # of full flights), ..., (Origin City, # of full flights)] 
def map_a_flights(df):
    res = []
    ratio = df["Passengers"]/df["Seats"]
    res.append([(df["Origin_city"][i], df["Flights"][i]) for i in df.index if (ratio[i] == 1)])
    return res 

#using res from above, we assign each Origin City to be a key in the default dict, and # of full flights to be it's value
def shuffle_a_flights(inpt): 
    d = defaultdict(list)
    for array in inpt : 
        for k, v in array:
            d[k].append(v)
    return d 

#finally, going through each key in our default dictionary, 
#we sum the number of full flights for all Origin Cities (and their values)
def reduce_a_flights(d): 
    res = []
    for k,v in d.items(): #could use d[0], d[1] but k=key, v=val more intuitive
        res.append([k, sum(v)]) 
    return res 
#format: 
#[[Origin City, sum(# of full flights)], ..., [Origin City, sum(# of full flights)]

#function that goes through all the Origin Cities, 
#finds the sum of all flights taken for this Origin City
#safely divides sum(# of full flights) sum(all flights from Origin City)
#l[combo][1] = Origin City; l[combo][1] = sum(# of full flights)
def percentage(df, l):
    for combo in range(len(l)):
        full_flights = l[combo][1] #sum(# of full flights) for each tuple
        all_flights = df.loc[df["Origin_city"] == l[combo][0] ]["Flights"].sum() #sum of all flights taken for this Origin City
        avg = np.divide(full_flights, all_flights, where=all_flights!=0) #avoid division by 0
        l[combo][1] = f"{np.round(avg*100, 2)}%"
    return l

In [109]:
small = percentage(df, reduce_a_flights(
            shuffle_a_flights(
                    map_a_flights( df ))))
medium = percentage(df_medium, reduce_a_flights(
            shuffle_a_flights(
                    map_a_flights( df_medium ))))
print("Small: ", small)
print("\n Medium: ", medium)

Small:  [['Elko, NV', '2.27%'], ['Seattle, WA', '0.02%'], ['Eugene, OR', '0.33%'], ['Portland, OR', '0.0%'], ['Duluth, MN', '11.11%'], ['Tulsa, OK', '3.92%'], ['Austin, TX', '6.9%'], ['Omaha, NE', '2.77%'], ['Wausau, WI', '14.44%'], ['Topeka, KS', '4.91%'], ['Peoria, IL', '9.88%'], ['Colorado Springs, CO', '7.14%'], ['Oklahoma City, OK', '9.9%'], ['Salt Lake City, UT', '0.03%'], ['North Platte, NE', '14.89%'], ['Spokane, WA', '4.71%'], ['Bismarck, ND', '15.79%'], ['Madison, WI', '10.61%'], ['Lincoln, NE', '10.87%'], ['Waterloo, IA', '22.86%'], ['Rockford, IL', '17.35%'], ['Green Bay, WI', '17.19%'], ['La Crosse, WI', '22.5%'], ['Milwaukee, WI', '7.3%'], ['Bellingham, WA', '6.63%'], ['Eau Claire, WI', '9.09%'], ['Mason City, IA', '33.33%'], ['Kansas City, MO', '5.31%'], ['San Antonio, TX', '5.35%'], ['Sioux Falls, SD', '18.18%'], ['Scottsbluff, NE', '5.88%'], ['Minneapolis, MN', '8.0%'], ['Cedar Rapids, IA', '10.83%'], ['Grand Island, NE', '5.26%'], ['Shreveport, LA', '20.0%'], ['Albuqu

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [47]:
# perhaps we could use combine for the full flights and "all" flights but this would be very expensive on RAM. 


### <strong> Bonus - Data Analysis </strong>
#### Data analysis

<p align="justify">
<font size="3">
Look at the number of almost empty flights and full flights and propose an analysis that explores if there is a correlation between the number of flights connecting two cities, the number of full flights and the percentage of almost empty flights. 

The granularity of the analysis is important: consider for example the week, the month and the year.
</font>
</p>


In [46]:
# Define here your considerations and your code